In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install ipywidgets
!pip install pandas
!git clone https://github.com/Chris1nexus/community-events.git
%cd community-events
!pip install .
%cd ..
!rm -rf community-events
!pip install wandb
#!huggingface-cli login
#!wandb login
#!accelerate config

In [ ]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os
import json
from huggingface_hub.hf_api import HfApi

hfapi = HfApi()
huggingnft_models = [model.modelId.split('/')[-1] for model in hfapi.list_models(author="huggingnft") if '__2__' in model.modelId.split('/')[-1] ]


chris1_models = [model.modelId.split('/')[-1] for model in hfapi.list_models(author="Chris1") if '__2__' in model.modelId.split('/')[-1] ]
created_models = set(huggingnft_models + chris1_models )

In [ ]:
to_ignore = ['sim2real_gta5_to_cityscapes']
hf_datasets_info = sorted( [datainfo.id for datainfo in hfapi.list_datasets(author="Chris1") \
                           if datainfo.id.split('/')[-1] not in to_ignore] )


cnt = 1
total = len(hf_datasets_info)* (len(hf_datasets_info) -1)//2
for idx1, hf_datapath1 in enumerate(hf_datasets_info):
    for idx2 in range(idx1+1, len(hf_datasets_info) )  :
        #hf_org = hf_datapath1.split('/')[0]
        hf_datapath2 = hf_datasets_info[idx2] 
        ds_name1 = hf_datapath1.split('/')[-1]
        ds_name2 = hf_datapath2.split('/')[-1]
        
        model_name = f'{ds_name1}__2__{ds_name2}'
        
        if model_name not in created_models:
            print(f'[{cnt }/{total}] - Creating {model_name}')
        else:
            print(f'[{cnt }/{total}] - Already created {model_name}')
        
        
        hyperparams_cyclegan = { 
                        'lr':[0.0002], 
                        'num_epochs':[100], 
                        'decay_epoch':[80] , #100 
                        'n_residual_blocks':[9], 
                        'lambda_cyc':[10.0], 
                        'lambda_id':[5.0], 
                        'beta1':[0.5], 'beta2':[0.999], 
                        'batch_size':[8], 
                        'source_dataset_name':[f'Chris1/{ds_name1}'],
                        'target_dataset_name':[f'Chris1/{ds_name2}'],
                        #'dataset_name':['Chris1/dooggies__2__nftrex'],#['Chris1/NFT_cryptopunks_to_bored_apes_HQ'],#['Chris1/NFT_cryptopunks_to_bored_apes'], 
                        'channels':[3], 
                        'checkpoint_interval':[5], #'cpu':[False], 
                        'epoch':[0], #'fp16':[False], 
                        'image_size':[256], #256
                        'mixed_precision':['no'], 
                        'num_workers':[8], 
                        'organization_name':['Chris1'], 
                        'wandb':[True],
                        'push_to_hub':[True], 
                        'model_name':['cyclegan'],
                        #'pytorch_dump_folder_path':[f'torch_dump_{ds_name1}__2__{ds_name2}'], 
                        'sample_interval':[10] }
        
        
        
        
        import copy
        import pandas as pd

        


        for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):
                
                EXPERIMENT_NAME = grid['model_name']
                copy_grid = copy.deepcopy(grid)
                if 'source_dataset_name' in copy_grid:
                    del copy_grid['source_dataset_name']
                if 'target_dataset_name' in copy_grid:
                    del copy_grid['target_dataset_name']
                if 'dataset' in copy_grid:
                    del copy_grid['dataset']
                if 'output_dir' in copy_grid:
                    del copy_grid['output_dir']

                call_params = ['accelerate', 'launch',
                               '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                               'train.py']

           
                grid['output_dir'] = 'experiments'

                for k,v in grid.items():


                    call_params.append(f'--{k}' )
                    if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                        call_params.append(f'{v}' )

                print(' '.join(call_params))
                call(call_params)

   
        
        
        
        
        
        
        
        cnt += 1

[Epoch 0/100] [Batch 356/541] [D loss: 0.228799] [G loss: 3.415998, adv: 0.900602, cycle: 0.176970, identity: 0.149139] ETA: 15:28:53.780552744706

In [ ]:
!accelerate launch --config_file ~/.cache/huggingface/accelerate/default_config.yaml train.py --batch_size 8 --beta1 0.5 --beta2 0.999 --channels 3 --checkpoint_interval 5 --decay_epoch 80 --epoch 0 --image_size 256 --lambda_cyc 10.0 --lambda_id 5.0 --lr 0.0002 --mixed_precision no --model_name cyclegan --n_residual_blocks 9 --num_epochs 100 --num_workers 8 --organization_name Chris1 --push_to_hub --sample_interval 10 --source_dataset_name Chris1/azuki --target_dataset_name Chris1/bored-ape-kennel-club --wandb --output_dir experiments

In [ ]:

d1 = 'boredapeyachtclub'
d2 = 'mutant-ape-yacht-club'
hyperparams_cyclegan = { 
                        'lr':[0.0002], 
                        'num_epochs':[200], 
                        'decay_epoch':[100] , #100 
                        'n_residual_blocks':[9], 
                        'lambda_cyc':[10.0], 
                        'lambda_id':[5.0], 
                        'beta1':[0.5], 'beta2':[0.999], 
                        'batch_size':[12], 
                        'source_dataset_name':[f'Chris1/{d1}'],
                        'target_dataset_name':[f'Chris1/{d2}'],
                        #'dataset_name':['Chris1/dooggies__2__nftrex'],#['Chris1/NFT_cryptopunks_to_bored_apes_HQ'],#['Chris1/NFT_cryptopunks_to_bored_apes'], 
                        'channels':[3], 
                        'checkpoint_interval':[5], #'cpu':[False], 
                        'epoch':[0], #'fp16':[False], 
                        'image_size':[256], #256
                        'mixed_precision':['no'], 
                        'num_workers':[8], 
                        'organization_name':['Chris1'], 
                        'wandb':[True],
                        'push_to_hub':[True], 
                        'model_name':[f'cyclegan_{d1}__2__{d2}'],
                        'pytorch_dump_folder_path':[f'torch_dump_{d1}__2__{d2}'], 
                        'sample_interval':[600] }


In [ ]:
import copy
import pandas as pd
EXPERIMENTS_DATA_CSV = 'experiments.csv'

if not os.path.exists(EXPERIMENTS_DATA_CSV):
    experiments_df = pd.DataFrame()
else:
    experiments_df = pd.read_csv(EXPERIMENTS_DATA_CSV)


EXPERIMENT_NAME = 'cyclegan_toadz__2__minimutants'
    

for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):

        copy_grid = copy.deepcopy(grid)
        if 'dataset_name' in copy_grid:
            del copy_grid['dataset_name']
        if 'dataset' in copy_grid:
            del copy_grid['dataset']
        if 'output_dir' in copy_grid:
            del copy_grid['output_dir']
            
        experiment_name =  '--'.join([f'{k}__{v}'  for k,v in copy_grid.items() ])
        call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
        
        
        experiment_id = len(experiments_df)
        grid['output_dir'] = experiment_name
        
        experiments_df = experiments_df.append(grid,ignore_index=True)
        grid['output_dir'] = f'experiments/{EXPERIMENT_NAME}/{experiment_id}'
        grid['pytorch_dump_folder_path'] = f'experiments/{grid["pytorch_dump_folder_path"]}_{experiment_id}'
        grid['model_name'] = f'{grid["model_name"]}_{experiment_id}'
        for k,v in grid.items():
            
             
            call_params.append(f'--{k}' )
            if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                call_params.append(f'{v}' )
                
        print(' '.join(call_params))
        call(call_params)
        
        
        experiments_df.to_csv(EXPERIMENTS_DATA_CSV)

In [ ]:
!accelerate launch --config_file ~/.cache/huggingface/accelerate/default_config.yaml train.py --batch_size 8 --beta1 0.5 --beta2 0.999 --channels 3 --checkpoint_interval 5 --decay_epoch 100 --epoch 0 --image_size 256 --lambda_cyc 10.0 --lambda_id 5.0 --lr 0.0002 --mixed_precision no --model_name test_cyclegan_toadz__2__minimutants_2 --n_residual_blocks 9 --num_epochs 200 --num_workers 8 --organization_name Chris1 --push_to_hub --pytorch_dump_folder_path experiments/torch_dump_toadz__2__minimutants_2 --sample_interval 30 --source_dataset_name Chris1/cryptopunks_HQ --target_dataset_name Chris1/bored_apes_yacht_club_HQ --wandb --output_dir experiments/cyclegan_toadz__2__minimutants/2

In [ ]:
!accelerate launch --config_file ~/.cache/huggingface/accelerate/default_config.yaml train.py --batch_size 8 --beta1 0.5 --beta2 0.999 --channels 3 --checkpoint_interval 5 --decay_epoch 100 --epoch 0 --image_size 256 --lambda_cyc 10.0 --lambda_id 5.0 --lr 0.0002 --mixed_precision no --model_name test_cyclegan_toadz__2__minimutants_3 --n_residual_blocks 9 --num_epochs 200 --num_workers 8 --organization_name Chris1 --push_to_hub --pytorch_dump_folder_path experiments/torch_dump_toadz__2__minimutants_3 --sample_interval 30 --source_dataset_name huggingnft/cryptoadz-by-gremplin --target_dataset_name huggingnft/mini-mutants --wandb --output_dir experiments/cyclegan_toadz__2__minimutants/3